In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import sklearn
import math
from scipy import linalg
from scipy.spatial.distance import cdist
import scipy.sparse.csgraph
import sklearn.neighbors
from scipy.sparse.linalg import eigs
import scipy.io
from scipy.sparse import csr_matrix
from PIL import Image
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))
import random

Current working directory: C:\Users\prateeksh\OneDrive - DPR Construction\Desktop\MS\Computational Data Analysis\Project


In [2]:
#reading the articles into a panda dataframe and converting it into a matrix
import pandas as pd
import random
p = 0.05  # 5% of the lines
# keep the header, then take only 5% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
data = pd.read_csv(
         'data/CNN.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
#data =  pd.read_csv('data/CNN.csv')
print(data)
data_updated = data.to_numpy()
print(data_updated)

                                             id  \
0      00128f1ba30d5e9e0f17df83285a1bc2072e2f01   
1      001ee59c375363263821474d40e4386ab91d5145   
2      00672d5c2055608b747a90a0f5ae32c5b340173e   
3      006bca7d18b3c6889ce567133566d22e491d27c1   
4      009e53253b0ba1823c4a08dfd6e4446ca9b02388   
...                                         ...   
14143  ffed6787eff6f1e01c8019328f5dcd53c697ad07   
14144  ffee67cf410c32857f263cb5b8a15b492194ac53   
14145  fff95a57a982743cc4752dc154a8f7ba701304a5   
14146  fffb34770ef63478d7f1154bc473f95f1efbc769   
14147  fffc82af3daa218f2e08e69ec325c2e1847a5490   

                                                 article  \
0      A woman has been charged with reckless manslau...   
1      These days we pick up a packet of frozen prawn...   
2      By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...   
3      Earlier this season I picked Thierry Henry as ...   
4      (CNN) -- Big-spending English club Manchester ...   
...                        

In [3]:
#reading all the list of topics excel sheets i.e. countries, politicians, hot political issues, US states
countries = pd.read_excel('data/Countries.xlsx')
politicians = pd.read_excel('data/Politicians.xlsx')
#issues = pd.read_excel('data/Hot Political Issues.xlsx')
#states = pd.read_excel('data/US States.xlsx')
  
countries_updated = countries.to_numpy()
politicians_updated = politicians.to_numpy()
#issues_updated = issues.to_numpy()
#states_updated = states.to_numpy()

print(countries_updated)
print(politicians_updated)
#print(issues_updated)
#print(states_updated)

[['United States']
 ['China']
 ['Japan']
 ['Germany']
 ['United Kingdom']
 ['India']
 ['France']
 ['Italy']
 ['Canada']
 ['South Korea']
 ['Russia']
 ['Brazil']
 ['Australia']
 ['Spain']
 ['Mexico']]
[['Biden']
 ['Trump']
 ['Sanders']
 ['Harris']
 ['Pence']
 ['Warren']
 ['Bloomberg']
 ['Buttigieg']
 ['Beto']
 ['Cheney']
 ['Cruz']
 ['DeSantis']
 ['Haley']
 ['Abott']
 ['Newsom']
 ['Noem']
 ['Cortez']
 ['Pompeo']]


In [4]:
#editing articles data matrix 'data_updated' to include indicator columns for individual topics in each list
#we will create an editied articles data matrix for each list of topic

data_countries = data.copy()
data_countries=pd.concat([data_countries,pd.DataFrame(columns=countries_updated[:,0].tolist())])
data_countries

,id,article,highlights,United States,China,Japan,Germany,United Kingdom,India,France,Italy,Canada,South Korea,Russia,Brazil,Australia,Spain,Mexico
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,"Claudia Yanira Hernandez Soriano, 25, and Juan...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,'I'll never eat a king prawn again' says Wicke...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...","Unveiling at 10am PST, 6pm GMT .\nInvitation i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Sportsmail columnist Martin Keown was honoured...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,English champion Manchester City posts loss of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14143,ffed6787eff6f1e01c8019328f5dcd53c697ad07,"By . Becky Evans . PUBLISHED: . 15:08 EST, 3 A...","PC Eileen Arthurs, 52, accessed the police nat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14144,ffee67cf410c32857f263cb5b8a15b492194ac53,(CNN) -- A late Sergio Ramos goal earned Real ...,Sergio Ramos headed goal earns Real Madrid sec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14145,fff95a57a982743cc4752dc154a8f7ba701304a5,By . Stephanie Linning . A reclusive pensioner...,Denis Walsby was found underneath stack of pap...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14146,fffb34770ef63478d7f1154bc473f95f1efbc769,"Much like Sam from the Dr. Seuss book, 'Green ...",'It tastes like cow poop!' said one young boy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#data_issues = data.copy()
#data_issues=pd.concat([data_issues,pd.DataFrame(columns=issues_updated[:,0].tolist())])
#data_issues

In [6]:
data_politicians = data.copy()
data_politicians=pd.concat([data_politicians,pd.DataFrame(columns=politicians_updated[:,0].tolist())])
data_politicians

,id,article,highlights,Biden,Trump,Sanders,Harris,Pence,Warren,Bloomberg,...,Beto,Cheney,Cruz,DeSantis,Haley,Abott,Newsom,Noem,Cortez,Pompeo
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,"Claudia Yanira Hernandez Soriano, 25, and Juan...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,'I'll never eat a king prawn again' says Wicke...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...","Unveiling at 10am PST, 6pm GMT .\nInvitation i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Sportsmail columnist Martin Keown was honoured...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,English champion Manchester City posts loss of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14143,ffed6787eff6f1e01c8019328f5dcd53c697ad07,"By . Becky Evans . PUBLISHED: . 15:08 EST, 3 A...","PC Eileen Arthurs, 52, accessed the police nat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14144,ffee67cf410c32857f263cb5b8a15b492194ac53,(CNN) -- A late Sergio Ramos goal earned Real ...,Sergio Ramos headed goal earns Real Madrid sec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14145,fff95a57a982743cc4752dc154a8f7ba701304a5,By . Stephanie Linning . A reclusive pensioner...,Denis Walsby was found underneath stack of pap...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14146,fffb34770ef63478d7f1154bc473f95f1efbc769,"Much like Sam from the Dr. Seuss book, 'Green ...",'It tastes like cow poop!' said one young boy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#data_states = data.copy()
#data_states=pd.concat([data_states,pd.DataFrame(columns=states_updated[:,0].tolist())])
#data_states

In [8]:
#counting number of instances of values of topics of each list in each article and setting the value of topic value column
#for the article as 1 if it the count exceeds a threshold value
import re
t = 1
data_countries_arr = data_countries.to_numpy()
for i in range(len(countries_updated)):
    word = countries_updated[i,0]
    for j in range(len(data_countries_arr)):
        string = data_countries_arr[j,1]
        c = len(re.findall(word, string,flags=re.IGNORECASE)) 
        if c >= t:
            data_countries[countries_updated[i,0]].values[j] = 1
        c = 0
data_countries

,id,article,highlights,United States,China,Japan,Germany,United Kingdom,India,France,Italy,Canada,South Korea,Russia,Brazil,Australia,Spain,Mexico
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,"Claudia Yanira Hernandez Soriano, 25, and Juan...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,'I'll never eat a king prawn again' says Wicke...,NaN,1,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...","Unveiling at 10am PST, 6pm GMT .\nInvitation i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Sportsmail columnist Martin Keown was honoured...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,English champion Manchester City posts loss of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14143,ffed6787eff6f1e01c8019328f5dcd53c697ad07,"By . Becky Evans . PUBLISHED: . 15:08 EST, 3 A...","PC Eileen Arthurs, 52, accessed the police nat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14144,ffee67cf410c32857f263cb5b8a15b492194ac53,(CNN) -- A late Sergio Ramos goal earned Real ...,Sergio Ramos headed goal earns Real Madrid sec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14145,fff95a57a982743cc4752dc154a8f7ba701304a5,By . Stephanie Linning . A reclusive pensioner...,Denis Walsby was found underneath stack of pap...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14146,fffb34770ef63478d7f1154bc473f95f1efbc769,"Much like Sam from the Dr. Seuss book, 'Green ...",'It tastes like cow poop!' said one young boy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


In [60]:
x = data_countries[data_countries['South Korea'] == 1]
x

,id,article,highlights,United States,China,Japan,Germany,United Kingdom,India,France,Italy,Canada,South Korea,Russia,Brazil,Australia,Spain,Mexico,Analysis,Polarity
16,015a8c6844420202dafaf8aa88e51f83c2747ab0,England is the only developed country producin...,'Deeply worrying' report shows scale of proble...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Positive,0.102401
80,05735dc02fed0630b660e140048d25ea556cffea,"Way back in the mists of time, when schoolkids...",Speculation about what baby boy will be called...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Positive,0.046262
308,145568a553d4a49a1084b391a93dc5a7fb6476c2,"(CNN) -- Usain Bolt, the anchor of Jamaican sp...","4x100m team of Carter, Frater, Blake and Bolt ...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Positive,0.078871
348,1675332735a8a6bda427c3dbe5579657861c5f99,(CNN) -- Poof. Gone. That's what will happen t...,Americans are expected to let two vacation day...,NaN,NaN,1,NaN,NaN,NaN,1,1,1,1,NaN,1,NaN,1,NaN,Positive,0.058647
486,1d9965d8042148b7854ba7c1377423402878079f,Craig Gordon insists it’s wrong to bill Sunday...,Rangers and Celtic will play each other in the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Positive,0.038258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13953,fa372ff9f45e6da581c7138cc5e19cdd80f71277,(CNN) -- Leandro Benitez scored both goals as ...,Leandro Benitez scores both goals as Estudiant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,Positive,0.001042
13958,fa583af33efc252f6dd9a58689dc1803f0d71202,Brazil World Cup winner Ronaldo was left to wa...,Spain's Rafael Nadal beat former Brazil strike...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,Negative,-0.001587
14025,fc2ade72243f8a9b6f6accada87fe9c3eca34ec1,"Seoul, South Korea (CNN) -- South Korea's fore...",Kim Sung-hwan says possibility being considere...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Positive,0.016250
14105,fedbd58e692f42367c274b8e61908bc5920b3136,North Koreans were treated to a exhibition of ...,Pro wrestling exhibition held in Pyongyang for...,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,Positive,0.007798


In [10]:
#counting number of instances of values of topics of each list in each article and setting the value of topic value column
#for the article as 1 if it the count exceeds a threshold value
import re
t = 2
data_politicians_arr = data_politicians.to_numpy()
for i in range(len(politicians_updated)):
    word = politicians_updated[i,0]
    for j in range(len(data_politicians_arr)):
        string = data_politicians_arr[j,1]
        c = len(re.findall(word, string,flags=re.IGNORECASE)) 
        if c >= t:
            data_politicians[politicians_updated[i,0]].values[j] = 1
        c = 0
data_politicians

,id,article,highlights,Biden,Trump,Sanders,Harris,Pence,Warren,Bloomberg,...,Beto,Cheney,Cruz,DeSantis,Haley,Abott,Newsom,Noem,Cortez,Pompeo
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,"Claudia Yanira Hernandez Soriano, 25, and Juan...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,'I'll never eat a king prawn again' says Wicke...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...","Unveiling at 10am PST, 6pm GMT .\nInvitation i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Sportsmail columnist Martin Keown was honoured...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,English champion Manchester City posts loss of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14143,ffed6787eff6f1e01c8019328f5dcd53c697ad07,"By . Becky Evans . PUBLISHED: . 15:08 EST, 3 A...","PC Eileen Arthurs, 52, accessed the police nat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14144,ffee67cf410c32857f263cb5b8a15b492194ac53,(CNN) -- A late Sergio Ramos goal earned Real ...,Sergio Ramos headed goal earns Real Madrid sec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14145,fff95a57a982743cc4752dc154a8f7ba701304a5,By . Stephanie Linning . A reclusive pensioner...,Denis Walsby was found underneath stack of pap...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14146,fffb34770ef63478d7f1154bc473f95f1efbc769,"Much like Sam from the Dr. Seuss book, 'Green ...",'It tastes like cow poop!' said one young boy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
#x = data_politicians[data_politicians['Biden'] == 1]
#x

,id,article,highlights,Biden,Trump,Sanders,Harris,Pence,Warren,Bloomberg,...,Cruz,DeSantis,Haley,Abott,Newsom,Noem,Cortez,Pompeo,Analysis,Polarity
905,3616ba0af1c490c8b596424b04bc7745c78ac60c,Vice President Joe Biden has become well known...,Vice President Joe Biden was seen getting very...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative,-0.054209
1184,45b8e099eebfd8a8c02f6b9a949c5c1ddab3cf69,"By . David Martosko, U.s. Political Editor . P...",Obama spokesman Jay Carney coyly told reporter...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.014117
2166,7cd8b95094a2a1ddfb523e9c01ca98112e379ef3,"By . Reuters . PUBLISHED: . 18:05 EST, 26 Nove...",Two unarmed U.S. B-52 bombers on a training mi...,1,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.083883
2328,87a6ec8b7a5e41c4eec57c56a0c2a7fbb4b893c7,By . Lee Moran . Last updated at 4:55 PM on 17...,U.S. Vice President warned against quantative ...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.076296
3922,e2e3fb9fab5d2e9473325695d744738b53223552,A U.S. Navy guided missile cruiser took evasiv...,U.S. Navy ship forced to come to an abrupt sto...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative,-0.008970
4035,e902305b5da40778914b341835249c121f5bce5e,(CNN) -- Like many immigrant parents of Centra...,Parents wait for weeks wondering if their chil...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.038733
4231,f451a7f84d63ca85e6334284c4d54385498ef4d7,At least two of the three men involved in a po...,Some officials linking threat to al Qaeda chie...,1,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.071202
4765,09aea3b860a68bc7c0ba716eab5ca1f75d43cd0d,"DORCHESTER, Massachusetts (CNN) -- Friends, f...","Joseph Kennedy: ""He had such a big heart, and ...",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.113727
4995,0fd634641984344d450b11d990250c53136f5de3,(CNN) -- Cheers in Moscow. Outrage in Kiev. Bl...,"NEW: Putin, Russia ""on the wrong side of histo...",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.004211
5333,18800465a68dd65a18bfd154b4ff4bfaa724671b,Washington (CNN) -- President Barack Obama on ...,"NEW: Decline in U.S. college degrees is ""a hug...",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.058452


In [12]:
#counting number of instances of values of topics of each list in each article and setting the value of topic value column
#for the article as 1 if it the count exceeds a threshold value
#import re
#t = 2
#data_issues_arr = data_issues.to_numpy()
#for i in range(len(issues_updated)):
    #word = issues_updated[i,0]
    #for j in range(len(data_issues_arr)):
        #string = data_issues_arr[j,1]
        #c = len(re.findall(word, string,flags=re.IGNORECASE)) 
        #if c >= t:
            #data_issues[issues_updated[i,0]].values[j] = 1
        #c = 0
#data_issues

In [13]:
#x = data_issues[data_issues['Abortion Rights'] == 1]
#x

In [14]:
#counting number of instances of values of topics of each list in each article and setting the value of topic value column
#for the article as 1 if it the count exceeds a threshold value
#import re
#t = 5
#data_states_arr = data_states.to_numpy()
#for i in range(len(states_updated)):
    #word = states_updated[i,0]
    #for j in range(len(data_states_arr)):
        #string = data_states_arr[j,1]
        #c = len(re.findall(word, string,flags=re.IGNORECASE)) 
        #if c >= t:
            #data_states[states_updated[i,0]].values[j] = 1
        #c = 0
#data_states

In [15]:
#x = data_states[data_states['Alabama'] == 1]
#x

In [16]:
articles = data_countries[['id', 'article']].copy()

In [17]:
# Define a function to clean the articles text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the article column
articles['cleaned articles'] = articles['article'].apply(clean)
articles.head()

,id,article,cleaned articles
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,A woman has been charged with reckless manslau...
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,These days we pick up a packet of frozen prawn...
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...",By Rob Waugh UPDATED EST September Apple has m...
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Earlier this season I picked Thierry Henry as ...
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,CNN Big spending English club Manchester City...


In [18]:
#import nltk
#nltk.download()

In [19]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

articles['POS tagged'] = articles['cleaned articles'].apply(token_stop_pos)
articles.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prateeksh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prateeksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prateeksh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,article,cleaned articles,POS tagged
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,A woman has been charged with reckless manslau...,"[(woman, n), (charged, v), (reckless, a), (man..."
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,These days we pick up a packet of frozen prawn...,"[(days, n), (pick, v), (packet, n), (frozen, a..."
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...",By Rob Waugh UPDATED EST September Apple has m...,"[(Rob, n), (Waugh, n), (UPDATED, n), (EST, n),..."
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Earlier this season I picked Thierry Henry as ...,"[(Earlier, r), (season, n), (picked, v), (Thie..."
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,CNN Big spending English club Manchester City...,"[(CNN, n), (Big, n), (spending, n), (English, ..."


In [20]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

articles['Lemma'] = articles['POS tagged'].apply(lemmatize)
articles.head()

,id,article,cleaned articles,POS tagged,Lemma
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,A woman has been charged with reckless manslau...,"[(woman, n), (charged, v), (reckless, a), (man...",woman charge reckless manslaughter boyfriend...
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,These days we pick up a packet of frozen prawn...,"[(days, n), (pick, v), (packet, n), (frozen, a...",day pick packet frozen prawn supermarket alm...
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...",By Rob Waugh UPDATED EST September Apple has m...,"[(Rob, n), (Waugh, n), (UPDATED, n), (EST, n),...",Rob Waugh UPDATED EST September Apple make l...
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Earlier this season I picked Thierry Henry as ...,"[(Earlier, r), (season, n), (picked, v), (Thie...",Earlier season pick Thierry Henry great ever...
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,CNN Big spending English club Manchester City...,"[(CNN, n), (Big, n), (spending, n), (English, ...",CNN Big spending English club Manchester Cit...


In [21]:
from textblob import TextBlob
# function to calculate subjectivity
def getSubjectivity(article):
    return TextBlob(article).sentiment.subjectivity

# function to calculate polarity
def getPolarity(article):
    return TextBlob(article).sentiment.polarity

# function to analyze the articles
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

# fin_data['Subjectivity'] = fin_data['Lemma'].apply(getSubjectivity) 
articles['Polarity'] = articles['Lemma'].apply(getPolarity) 
articles['Analysis'] = articles['Polarity'].apply(analysis)
articles.head()

,id,article,cleaned articles,POS tagged,Lemma,Polarity,Analysis
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,A woman has been charged with reckless manslau...,"[(woman, n), (charged, v), (reckless, a), (man...",woman charge reckless manslaughter boyfriend...,0.025455,Positive
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,These days we pick up a packet of frozen prawn...,"[(days, n), (pick, v), (packet, n), (frozen, a...",day pick packet frozen prawn supermarket alm...,-0.005938,Negative
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...",By Rob Waugh UPDATED EST September Apple has m...,"[(Rob, n), (Waugh, n), (UPDATED, n), (EST, n),...",Rob Waugh UPDATED EST September Apple make l...,0.109265,Positive
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Earlier this season I picked Thierry Henry as ...,"[(Earlier, r), (season, n), (picked, v), (Thie...",Earlier season pick Thierry Henry great ever...,0.345833,Positive
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,CNN Big spending English club Manchester City...,"[(CNN, n), (Big, n), (spending, n), (English, ...",CNN Big spending English club Manchester Cit...,0.066291,Positive


In [22]:
#data_states['Analysis'] = articles['Analysis']
#data_states['Polarity'] = articles['Polarity']
data_countries['Analysis'] = articles['Analysis']
data_countries['Polarity'] = articles['Polarity']
#data_issues['Analysis'] = articles['Analysis']
#data_issues['Polarity'] = articles['Polarity']
data_politicians['Analysis'] = articles['Analysis']
data_politicians['Polarity'] = articles['Polarity']

In [52]:
#data_politicians.head()
data_countries

,id,article,highlights,United States,China,Japan,Germany,United Kingdom,India,France,Italy,Canada,South Korea,Russia,Brazil,Australia,Spain,Mexico,Analysis,Polarity
0,00128f1ba30d5e9e0f17df83285a1bc2072e2f01,A woman has been charged with reckless manslau...,"Claudia Yanira Hernandez Soriano, 25, and Juan...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.025455
1,001ee59c375363263821474d40e4386ab91d5145,These days we pick up a packet of frozen prawn...,'I'll never eat a king prawn again' says Wicke...,NaN,1,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative,-0.005938
2,00672d5c2055608b747a90a0f5ae32c5b340173e,"By . Rob Waugh . UPDATED: . 04:54 EST, 28 Sept...","Unveiling at 10am PST, 6pm GMT .\nInvitation i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.109265
3,006bca7d18b3c6889ce567133566d22e491d27c1,Earlier this season I picked Thierry Henry as ...,Sportsmail columnist Martin Keown was honoured...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.345833
4,009e53253b0ba1823c4a08dfd6e4446ca9b02388,(CNN) -- Big-spending English club Manchester ...,English champion Manchester City posts loss of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.066291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14143,ffed6787eff6f1e01c8019328f5dcd53c697ad07,"By . Becky Evans . PUBLISHED: . 15:08 EST, 3 A...","PC Eileen Arthurs, 52, accessed the police nat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.021780
14144,ffee67cf410c32857f263cb5b8a15b492194ac53,(CNN) -- A late Sergio Ramos goal earned Real ...,Sergio Ramos headed goal earns Real Madrid sec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,0.096545
14145,fff95a57a982743cc4752dc154a8f7ba701304a5,By . Stephanie Linning . A reclusive pensioner...,Denis Walsby was found underneath stack of pap...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative,-0.007061
14146,fffb34770ef63478d7f1154bc473f95f1efbc769,"Much like Sam from the Dr. Seuss book, 'Green ...",'It tastes like cow poop!' said one young boy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,Positive,0.033207


In [24]:
#spectral analysis for politicians
#creating adjacency matrix
n,m = data_politicians.shape
A = np.empty((n, n))
D = np.empty((n, n))

data_politicians_arr = data_politicians.to_numpy()
for i in range(len(data_politicians_arr)):
    for j in range(len(data_politicians_arr)):
        topic_count = 0
        if i == j:
            continue
        else:
            for k in range(3,m-2):
                if data_politicians_arr[i,k] == 'NaN':
                    continue
                elif data_politicians_arr[i,k] == data_politicians_arr[j,k]:
                    topic_count = topic_count + 1
        A[i][j] = topic_count


In [25]:
print(A)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [30]:
#B = np.where(A[1000:2000,:] != 0)
#print(B)

(array([ 65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
        65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
        65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
        65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
        65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
        65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
        65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
        65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
       166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166,
       166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166,
       166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166,
       166, 166, 166, 166, 166, 166, 166, 166, 184, 184, 184, 184, 184,
       184, 184, 184, 184, 184, 184, 184, 184, 184, 184, 184, 184, 184,
       184, 184, 184, 184, 184, 184, 184, 184, 202, 202, 202, 2

In [31]:
D = np.diag(A.sum(axis=1))
print(D)
L = D-A
print(L)

from numpy import linalg as LA
from sklearn.cluster import KMeans

w, v = LA.eigh(L)
# w and v are eignevalue vector and eignevectors matrix#
print(w)
print(v)
print(w.shape)
print(v.shape)

kmeans = KMeans(n_clusters=4)
kmeans.fit(v[:,0:3])
labels = kmeans.labels_
print(labels.shape)
print(labels)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[-1.14462781e-13 -1.09141038e-13 -1.01954257e-13 ...  1.05000000e+02
  1.05000000e+02  1.05000000e+02]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(14148,)
(14148, 14148)
(14148,)
[0 0 0 ... 0 0 0]


In [28]:
#spectral analysis for countries
#creating adjacency matrix
n2,m2 = data_countries.shape
A2 = np.empty((n2, n2))
D2 = np.empty((n2, n2))

data_countries_arr = data_countries.to_numpy()
for i in range(len(data_countries_arr)):
    for j in range(len(data_countries_arr)):
        topic_count = 0
        if i == j:
            continue
        else:
            for k in range(3,m2-2):
                if data_countries_arr[i,k] == 'NaN':
                    continue
                elif data_countries_arr[i,k] == data_countries_arr[j,k]:
                    topic_count = topic_count + 1
        A2[i][j] = topic_count

In [32]:
print(A2)
D2 = np.diag(A2.sum(axis=1))
print(D2)
L2 = D2-A2
print(L2)

from numpy import linalg as LA
from sklearn.cluster import KMeans

w2, v2 = LA.eigh(L2)
# w and v are eignevalue vector and eignevectors matrix#
print(w2)
print(v2)
print(w2.shape)
print(v2.shape)

kmeans = KMeans(n_clusters=4)
kmeans.fit(v2[:,0:3])
labels2 = kmeans.labels_
print(labels2.shape)
print(labels2)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[   0.    0.    0. ...    0.    0.    0.]
 [   0. 2099.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ...  614.    0.    0.]
 [   0.    0.    0. ...    0.  491.    0.]
 [   0.    0.    0. ...    0.    0.    0.]]
[[   0.    0.    0. ...    0.    0.    0.]
 [   0. 2099.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ...  614.    0.    0.]
 [   0.    0.    0. ...    0.  491.    0.]
 [   0.    0.    0. ...    0.    0.    0.]]
[-4.88421658e-12 -4.76796376e-12 -4.68681488e-12 ...  5.94094204e+03
  6.27347652e+03  6.60374197e+03]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.11895752e-04
  -5.32120988e-04 -4.87464666e-04]
 [-

In [40]:
print(np.sum(labels == 0))
print(np.sum(labels == 1))
print(np.sum(labels == 2))
print(np.sum(labels == 3))

13576
177
173
222


In [41]:
print(np.sum(labels2 == 0))
print(np.sum(labels2 == 1))
print(np.sum(labels2 == 2))
print(np.sum(labels2 == 3))

12990
528
139
491


In [49]:
#calculating majority sentiment label for each cluster for politicians
k = 4
major_label=np.empty(k, dtype=object)
mismatch_rate=np.empty(k, dtype=float)
for i in range(k):
    sum_senti_label = 0
    count = 0
    mismatch_count = 0
    for j in range(len(labels)):
        if labels[j] == i:
            if data_politicians_arr[j,21] == 'Positive':
                sum_senti_label = sum_senti_label + 1
            count = count + 1
    if sum_senti_label >= (count/2):
        major_label[i] = 'Positive'
    else:
        major_label[i] = 'Negative'
    for j in range(len(labels)):
        if labels[j] == i:
            if data_politicians_arr[j,21] != major_label[i]:
                mismatch_count = mismatch_count + 1
    mismatch_rate[i] = ((mismatch_count/count) * 100)
    print("The majority sentiment for cluster " + str(i) + " is " + str(major_label[i]))
    print("The mismatch rate for cluster " + str(i) + " is " + str(mismatch_rate[i]))

print(major_label)
print(mismatch_rate)

The majority sentiment for cluster 0 is Positive
The mismatch rate for cluster 0 is 21.33176193282263
The majority sentiment for cluster 1 is Positive
The mismatch rate for cluster 1 is 27.683615819209038
The majority sentiment for cluster 2 is Positive
The mismatch rate for cluster 2 is 17.341040462427745
The majority sentiment for cluster 3 is Positive
The mismatch rate for cluster 3 is 20.72072072072072
['Positive' 'Positive' 'Positive' 'Positive']
[21.33176193 27.68361582 17.34104046 20.72072072]


In [57]:
#calculating majority sentiment label for each cluster for countries
k = 4
major_label2=np.empty(k, dtype=object)
mismatch_rate2=np.empty(k, dtype=float)
for i in range(k):
    sum_senti_label = 0
    count = 0
    mismatch_count = 0
    for j in range(len(labels2)):
        if labels2[j] == i:
            if data_countries_arr[j,18] == 'Positive':
                sum_senti_label = sum_senti_label + 1
            count = count + 1
    if sum_senti_label >= (count/2):
        major_label2[i] = 'Positive'
    else:
        major_label2[i] = 'Negative'
    for j in range(len(labels2)):
        if labels2[j] == i:
            if data_countries_arr[j,18] != major_label2[i]:
                mismatch_count = mismatch_count + 1
    mismatch_rate2[i] = ((mismatch_count/count) * 100)
    print("The majority sentiment for cluster " + str(i) + " is " + str(major_label2[i]))
    print("The mismatch rate for cluster " + str(i) + " is " + str(mismatch_rate2[i]))

print(major_label2)
print(mismatch_rate2)

The majority sentiment for cluster 0 is Positive
The mismatch rate for cluster 0 is 20.939483707151926
The majority sentiment for cluster 1 is Positive
The mismatch rate for cluster 1 is 21.84873949579832
The majority sentiment for cluster 2 is Positive
The mismatch rate for cluster 2 is 26.36986301369863
The majority sentiment for cluster 3 is Positive
The mismatch rate for cluster 3 is 22.093023255813954
['Positive' 'Positive' 'Positive' 'Positive']
[20.93948371 21.8487395  26.36986301 22.09302326]


In [54]:
#repeating the analysis for k = 10
D = np.diag(A.sum(axis=1))
print(D)
L = D-A
print(L)

from numpy import linalg as LA
from sklearn.cluster import KMeans

w, v = LA.eigh(L)
# w and v are eignevalue vector and eignevectors matrix#
print(w)
print(v)
print(w.shape)
print(v.shape)

kmeans = KMeans(n_clusters=10)
kmeans.fit(v[:,0:9])
labels = kmeans.labels_
print(labels.shape)
print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[-1.14462781e-13 -1.09141038e-13 -1.01954257e-13 ...  1.05000000e+02
  1.05000000e+02  1.05000000e+02]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(14148,)
(14148, 14148)
(14148,)
[0 0 0 ... 0 0 0]


In [55]:
print(A2)
D2 = np.diag(A2.sum(axis=1))
print(D2)
L2 = D2-A2
print(L2)

from numpy import linalg as LA
from sklearn.cluster import KMeans

w2, v2 = LA.eigh(L2)
# w and v are eignevalue vector and eignevectors matrix#
print(w2)
print(v2)
print(w2.shape)
print(v2.shape)

kmeans = KMeans(n_clusters=10)
kmeans.fit(v2[:,0:9])
labels2 = kmeans.labels_
print(labels2.shape)
print(labels2)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[   0.    0.    0. ...    0.    0.    0.]
 [   0. 2099.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ...  614.    0.    0.]
 [   0.    0.    0. ...    0.  491.    0.]
 [   0.    0.    0. ...    0.    0.    0.]]
[[   0.    0.    0. ...    0.    0.    0.]
 [   0. 2099.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 ...
 [   0.    0.    0. ...  614.    0.    0.]
 [   0.    0.    0. ...    0.  491.    0.]
 [   0.    0.    0. ...    0.    0.    0.]]
[-4.88421658e-12 -4.76796376e-12 -4.68681488e-12 ...  5.94094204e+03
  6.27347652e+03  6.60374197e+03]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.11895752e-04
  -5.32120988e-04 -4.87464666e-04]
 [-

In [56]:
#calculating majority sentiment label for each cluster for politicians
k = 10
major_label=np.empty(k, dtype=object)
mismatch_rate=np.empty(k, dtype=float)
for i in range(k):
    sum_senti_label = 0
    count = 0
    mismatch_count = 0
    for j in range(len(labels)):
        if labels[j] == i:
            if data_politicians_arr[j,21] == 'Positive':
                sum_senti_label = sum_senti_label + 1
            count = count + 1
    if sum_senti_label >= (count/2):
        major_label[i] = 'Positive'
    else:
        major_label[i] = 'Negative'
    for j in range(len(labels)):
        if labels[j] == i:
            if data_politicians_arr[j,21] != major_label[i]:
                mismatch_count = mismatch_count + 1
    mismatch_rate[i] = ((mismatch_count/count) * 100)
    print("The majority sentiment for cluster " + str(i) + " is " + str(major_label[i]))
    print("The mismatch rate for cluster " + str(i) + " is " + str(mismatch_rate[i]))

print(major_label)
print(mismatch_rate)

The majority sentiment for cluster 0 is Positive
The mismatch rate for cluster 0 is 21.308325005765237
The majority sentiment for cluster 1 is Positive
The mismatch rate for cluster 1 is 18.75
The majority sentiment for cluster 2 is Positive
The mismatch rate for cluster 2 is 19.718309859154928
The majority sentiment for cluster 3 is Positive
The mismatch rate for cluster 3 is 22.36842105263158
The majority sentiment for cluster 4 is Positive
The mismatch rate for cluster 4 is 9.090909090909092
The majority sentiment for cluster 5 is Positive
The mismatch rate for cluster 5 is 22.52252252252252
The majority sentiment for cluster 6 is Positive
The mismatch rate for cluster 6 is 25.297619047619047
The majority sentiment for cluster 7 is Positive
The mismatch rate for cluster 7 is 18.446601941747574
The majority sentiment for cluster 8 is Positive
The mismatch rate for cluster 8 is 7.6923076923076925
The majority sentiment for cluster 9 is Positive
The mismatch rate for cluster 9 is 25.0


In [64]:
print(np.sum(labels == 0))
print(np.sum(labels == 1))
print(np.sum(labels == 2))
print(np.sum(labels == 3))
print(np.sum(labels == 4))
print(np.sum(labels == 5))
print(np.sum(labels == 6))
print(np.sum(labels == 7))
print(np.sum(labels == 8))
print(np.sum(labels == 9))

13009
64
71
76
11
111
336
309
13
148


In [58]:
#calculating majority sentiment label for each cluster for countries
k = 10
major_label2=np.empty(k, dtype=object)
mismatch_rate2=np.empty(k, dtype=float)
for i in range(k):
    sum_senti_label = 0
    count = 0
    mismatch_count = 0
    for j in range(len(labels2)):
        if labels2[j] == i:
            if data_countries_arr[j,18] == 'Positive':
                sum_senti_label = sum_senti_label + 1
            count = count + 1
    if sum_senti_label >= (count/2):
        major_label2[i] = 'Positive'
    else:
        major_label2[i] = 'Negative'
    for j in range(len(labels2)):
        if labels2[j] == i:
            if data_countries_arr[j,18] != major_label2[i]:
                mismatch_count = mismatch_count + 1
    mismatch_rate2[i] = ((mismatch_count/count) * 100)
    print("The majority sentiment for cluster " + str(i) + " is " + str(major_label2[i]))
    print("The mismatch rate for cluster " + str(i) + " is " + str(mismatch_rate2[i]))

print(major_label2)
print(mismatch_rate2)

The majority sentiment for cluster 0 is Positive
The mismatch rate for cluster 0 is 20.939483707151926
The majority sentiment for cluster 1 is Positive
The mismatch rate for cluster 1 is 21.84873949579832
The majority sentiment for cluster 2 is Positive
The mismatch rate for cluster 2 is 26.36986301369863
The majority sentiment for cluster 3 is Positive
The mismatch rate for cluster 3 is 22.093023255813954
The majority sentiment for cluster 4 is Positive
The mismatch rate for cluster 4 is 25.336322869955158
The majority sentiment for cluster 5 is Positive
The mismatch rate for cluster 5 is 18.75
The majority sentiment for cluster 6 is Positive
The mismatch rate for cluster 6 is 22.590361445783135
The majority sentiment for cluster 7 is Positive
The mismatch rate for cluster 7 is 25.671641791044774
The majority sentiment for cluster 8 is Positive
The mismatch rate for cluster 8 is 19.475655430711612
The majority sentiment for cluster 9 is Positive
The mismatch rate for cluster 9 is 25.5

In [65]:
print(np.sum(labels2 == 0))
print(np.sum(labels2 == 1))
print(np.sum(labels2 == 2))
print(np.sum(labels2 == 3))
print(np.sum(labels2 == 4))
print(np.sum(labels2 == 5))
print(np.sum(labels2 == 6))
print(np.sum(labels2 == 7))
print(np.sum(labels2 == 8))
print(np.sum(labels2 == 9))

11815
357
292
86
446
128
332
335
267
90
